In [1]:
import pandas as pd
import numpy as np
import findspark
findspark.init('C:\spark\spark-3.1.2-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import col
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


In [20]:
df = spark.read.csv("zoo.csv", header=True, inferSchema=True)

In [21]:
df.show()

+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+
|AnimalName|Hair|Feathers|Eggs|Milk|Airborne|Aquatic|Predator|Toothed|Backbone|Breathes|Venomous|Fins|Legs|Tail|Domestic|Catsize|Type|
+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+
|  aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|
|  antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|   1|
|      bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|   4|
|      bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|
|      boar|   1|       0|   0|   1|       0|      0|  

In [ ]:
from pyspark.sql.functions import col,when

In [23]:
df2 = df.withColumn("IsMammal", when(col("Type") ==1,"True").otherwise("False"))

In [24]:
df2.show()

+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+--------+
|AnimalName|Hair|Feathers|Eggs|Milk|Airborne|Aquatic|Predator|Toothed|Backbone|Breathes|Venomous|Fins|Legs|Tail|Domestic|Catsize|Type|IsMammal|
+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+--------+
|  aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|    True|
|  antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|   1|    True|
|      bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|   4|   False|
|      bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|  

In [25]:
df2.select("Type","IsMammal").show()

+----+--------+
|Type|IsMammal|
+----+--------+
|   1|    True|
|   1|    True|
|   4|   False|
|   1|    True|
|   1|    True|
|   1|    True|
|   1|    True|
|   4|   False|
|   4|   False|
|   1|    True|
|   1|    True|
|   2|   False|
|   4|   False|
|   7|   False|
|   7|   False|
|   7|   False|
|   2|   False|
|   1|    True|
|   4|   False|
|   1|    True|
+----+--------+
only showing top 20 rows



In [26]:
from pyspark.ml.feature import RFormula

In [29]:
supervised = RFormula(formula = "IsMammal ~ . + AnimalName:Hair + AnimalName:Feathers + AnimalName:Eggs + AnimalName:Milk + AnimalName:Airborne + AnimalName:Aquatic + AnimalName:Predator + AnimalName:Toothed + AnimalName:Backbone + AnimalName:Breathes + AnimalName:Venomous + AnimalName:Fins + AnimalName:Legs + AnimalName:Tail + AnimalName:Domestic + AnimalName:Catsize + AnimalName:Type")

In [30]:
fittedRF = supervised.fit(df2)
preparedDF = fittedRF.transform(df2)
preparedDF.show()

+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+--------+--------------------+-----+
|AnimalName|Hair|Feathers|Eggs|Milk|Airborne|Aquatic|Predator|Toothed|Backbone|Breathes|Venomous|Fins|Legs|Tail|Domestic|Catsize|Type|IsMammal|            features|label|
+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+--------+--------------------+-----+
|  aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|    True|(1816,[1,99,102,1...|  1.0|
|  antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|   1|    True|(1816,[2,99,102,1...|  1.0|
|      bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|   4|   False|(1

In [31]:
train,test = preparedDF.randomSplit([0.7,0.3])

In [33]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol = "label",featuresCol = "features")

In [34]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

In [35]:
fittedLR = lr.fit(train)

In [44]:
fittedLR.transform(train).select("AnimalName","IsMammal","label","prediction").show(100)

+----------+--------+-----+----------+
|AnimalName|IsMammal|label|prediction|
+----------+--------+-----+----------+
|  antelope|    True|  1.0|       1.0|
|      bass|   False|  0.0|       0.0|
|      bear|    True|  1.0|       1.0|
|      boar|    True|  1.0|       1.0|
|   buffalo|    True|  1.0|       1.0|
|      calf|    True|  1.0|       1.0|
|      carp|   False|  0.0|       0.0|
|   catfish|   False|  0.0|       0.0|
|      cavy|    True|  1.0|       1.0|
|   chicken|   False|  0.0|       0.0|
|      chub|   False|  0.0|       0.0|
|      crab|   False|  0.0|       0.0|
|  crayfish|   False|  0.0|       0.0|
|      crow|   False|  0.0|       0.0|
|      deer|    True|  1.0|       1.0|
|   dolphin|    True|  1.0|       1.0|
|      dove|   False|  0.0|       0.0|
|      duck|   False|  0.0|       0.0|
|  flamingo|   False|  0.0|       0.0|
|      flea|   False|  0.0|       0.0|
|      frog|   False|  0.0|       0.0|
|   giraffe|    True|  1.0|       1.0|
|      gnat|   False|  0.